In [5]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch
import random
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupKFold

from scipy.signal import ShortTimeFFT
from scipy.signal.windows import gaussian

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
from constants import region_to_channel_dict, regions

/user/home/ko20929/miniconda3/envs/sktime_latest/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package yasa is out of date. Your version is 0.6.3, the latest is 0.6.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [31]:
from sklearn.linear_model import LinearRegression

In [41]:
df[['Channels']]

,Channels
0,14
1,14
2,14
3,14
4,14
5,7
6,1
7,1
8,1
9,1


In [39]:
df.columns

Index(['Channels', 'Segment Seconds', 'Batch Size', 'Att Depth', 'Att Heads',
       'GPU Usage (GB)'],
      dtype='object')

In [42]:
file_path = 'GPU_Memory_Usage_Stats .xlsx'
df = pd.read_excel(file_path)
df = df.drop(['Data Type', 'Segment Length', 'Batch Alloc (GB) '], axis=1)
channels = df[['Channels']]

df = df.drop(['Channels'], axis = 1)

# Filtering the DataFrame for rows where 'GPU Usage (GB)' is 'OOM'
oom_rows = df[df['GPU Usage (GB)'] == 'OOM']
oom_rows = oom_rows.apply(pd.to_numeric, errors='coerce')

training_rows = df[df['GPU Usage (GB)'] != 'OOM']
training_rows = training_rows.apply(pd.to_numeric, errors='coerce')
training_rows

,Segment Seconds,Batch Size,Att Depth,Att Heads,GPU Usage (GB)
0,120.000000,2,6.0,10.0,5.800
1,120.000000,2,6.0,5.0,3.200
2,120.000000,2,3.0,10.0,3.700
3,59.960938,2,3.0,10.0,1.304
4,59.960938,2,3.0,10.0,1.434
5,120.000000,2,6.0,10.0,5.800
6,120.000000,2,6.0,10.0,5.840
7,120.000000,2,6.0,10.0,5.840
10,120.000000,8,3.0,5.0,6.980


In [46]:
X = training_rows.iloc[:,:-1]
y = training_rows.iloc[:,-1]
reg = LinearRegression().fit(X, y)
print( reg.score(X, y) )
reg.coef_

0.9997130711498786


array([0.03882472, 0.98333333, 0.70666667, 0.524     ])

In [51]:
reg.predict(oom_rows[X.columns].iloc[:2])

array([11.72,  9.1 ])

In [23]:
training_rows.columns

Index(['Channels', 'Data Type', 'Segment Length', 'Segment Seconds',
       'Batch Size', 'Att Depth', 'Att Heads', 'GPU Usage (GB)',
       'Batch Alloc (GB) '],
      dtype='object')

In [59]:
reg.predict(concated_df[X.columns].iloc[:-1])

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [62]:
concated_df.columns 

Index(['Segment Seconds', 'Batch Size', 'Att Depth', 'Att Heads',
       'GPU Usage (GB)', 'predics'],
      dtype='object')

In [61]:
concated_df = pd.concat([training_rows , oom_rows], axis = 0)

concated_df['predics'] =  np.append(reg.predict(concated_df[X.columns].iloc[:-1]), np.nan)

concated_df

,Segment Seconds,Batch Size,Att Depth,Att Heads,GPU Usage (GB),predics
0,120.000000,2,6.0,10.0,5.800,5.820
1,120.000000,2,6.0,5.0,3.200,3.200
2,120.000000,2,3.0,10.0,3.700,3.700
3,59.960938,2,3.0,10.0,1.304,1.369
4,59.960938,2,3.0,10.0,1.434,1.369
5,120.000000,2,6.0,10.0,5.800,5.820
6,120.000000,2,6.0,10.0,5.840,5.820
7,120.000000,2,6.0,10.0,5.840,5.820
10,120.000000,8,3.0,5.0,6.980,6.980
8,120.000000,8,6.0,10.0,NaN,11.720


In [63]:
X.columns

Index(['Segment Seconds', 'Batch Size', 'Att Depth', 'Att Heads'], dtype='object')

In [80]:
dict_to_check = {'Segment Seconds' : [4] , 'Batch Size' : [16] , 'Att Depth' : [6] , 'Att Heads' : [4] }
dict_to_check_df = pd.DataFrame(dict_to_check)
reg.predict(dict_to_check_df)

array([11.93899874])

In [81]:
11.9 * 1.8 

21.42